In [4]:
from simulation_functions import index, simulation_, plot
from interpolation_functions import T_K
import numpy as np
import matplotlib.pyplot as plt

t_drive_new = np.loadtxt('t_drive_new.txt') # true time sample.
ind = index(15,t_drive_new) # 15 means that the time delay is chosen randomly between 0 and 15 days

# getting the indices of each time shifted curve
index0, index1,index2,index3 = ind[0],ind[1],ind[2],ind[3]
sample = np.sort(ind[5])

#getting the time sample %run %run %run of each curve
ts = np.array([sample[index0],sample[index1],sample[index2],sample[index3]])


# finding the magnitudes corresponding to each time shifted sample
f = simulation_(sample) #running simulation
fs = np.array([f[index0],f[index1],f[index2],f[index3]])

# desired sampling for each curve
tk = np.array([T_K(3,ts[0])[0],T_K(3,ts[1])[0],T_K(3,ts[2])[0],T_K(3,ts[3])[0]]) # desired sampling is 

h=3
time_delays = ind[6]

# sinc matrix
matrix = []
for i in range(4):
    matrix.append(np.array([(ts[i][:,np.newaxis]+time_delays[i])-tk[i][np.newaxis,:]]))
matrix = np.array(matrix)

A = np.sinc(np.concatenate([matrix[0],matrix[1],matrix[2],matrix[3]],axis =0)/h)

Y= np.concatenate([fs[0][np.newaxis,:],fs[1][np.newaxis,:],fs[2][np.newaxis,:],fs[3][np.newaxis,:]])

In [8]:
# multiplication functions:
noise = np.array([0.005887229046789439,0.009518036675569332,0.017490923069771385,0.04275225621685116]) # from running make_noise.py
def AT(X,A,Y): 
    '''
    calculateS the weighted mean of A.T(Y-AX)
    input:
    X : array with shape k, solution of interpolation
    A : tensor of shape 4xkxs, sinc matrix
    Y : matrix of shape 4xs, magnitudes of original time sampling
    '''
    weighted_mean = 0
    for i in range(4):
        weighted_mean += ((1/noise[i])**2)*np.dot(A[i].T, Y[i] - A[i] @ X)/np.sum(1/(noise**2))
    
    return weighted_mean

def multiplication(X,S): # X.S
    '''
    multiplies X by S[i] and takes the arithmetic mean
    '''
    this =[]
    for i in range(4):
        this.append(np.dot(X,S[i]))
    this = np.array(this)
    return np.mean(this,axis=0)

def multiplication_T(X,S): # X.S.T

    '''
    multiplies X by S[i].T and take the arithmetic mean
    '''
    this =[]
    for i in range(4):
        this.append(np.dot(X,S[i].T))
    this = np.array(this)
    return np.mean(this,axis=0)

In [9]:
def linorm(S, nit):
    x0 = np.random.rand(S.shape[1])  # chose [1] per the implementation of the power iteration method in wiki
    x0 = x0 /np.linalg.norm(x0)
    for i in range(nit):
        x = multiplication(x0,S)#np.dot(x0,S)
        xn = np.linalg.norm(x)
        xp = x / xn#     
        y = multiplication_T(xp,S) # the S.T is done within the functionas S[i].T not S.T[i]
        yn = np.linalg.norm(y)  
        x0 = y / yn
    return 1./xn
mu = linorm(A,20)/170 # = 0.0038

X = np.zeros(len(tk[0]))

count = 0
R = [np.sum(Y ** 2)]
epsilon = 0.3



while (R[-1] > epsilon) and (
        count < 5000):  # 5000 is a good amount of counts
    X_new = X + mu * AT(X,A,Y)
    X = X_new.copy()
    R.append(np.sum((Y - A @ X) ** 2.))
    count += 1


plt.title('Convergence')
plt.plot(np.array(R[1:]))
plt.xlabel('iterations')
plt.ylabel('error')
plt.show()




In [10]:
%matplotlib qt
cut = np.logical_and(X < np.max(Y), X > np.min(Y))
plt.plot(ts[0], Y[0], 'o', label='light curve 1')
plt.plot(ts[1], Y[1], 'o', label='light curve 2')
plt.plot(ts[2], Y[2], 'o', label='light curve 3')
plt.plot(ts[3], Y[3], 'o', label='light curve 4')

plt.plot(tk[0][cut], X[cut],'o', label='interpolated', color='black')
plt.xlabel('time[days]')
plt.ylabel('magnitude')
plt.title('sinc interpolated values')
plt.legend()
plt.show()

wondering about: (to discuss in meeting)
- multiplication_T(X,S): X,S[i].T versus X,S.T[i]
- interpolation is not that good, X from first curve being input to second curves seems to interpolate better.